# <p style="text-align:center"> Limpieza de datos </p>

Importamos las librerías que vamos a utilizar

In [1]:
import pandas as pd
import numpy as np

Importamos el csv de datos

In [2]:
uncleansed_df = pd.read_csv('../airbnb-listings.csv', delimiter=";", low_memory=False)
# El df original está separado por ; para que pueda leerlo bien especifico con delimiter=';' 

Lectura de las columnas del dataframe

In [292]:
print(uncleansed_df.columns.tolist())

['ID', 'Listing Url', 'Scrape ID', 'Last Scraped', 'Name', 'Summary', 'Space', 'Description', 'Experiences Offered', 'Neighborhood Overview', 'Notes', 'Transit', 'Access', 'Interaction', 'House Rules', 'Thumbnail Url', 'Medium Url', 'Picture Url', 'XL Picture Url', 'Host ID', 'Host URL', 'Host Name', 'Host Since', 'Host Location', 'Host About', 'Host Response Time', 'Host Response Rate', 'Host Acceptance Rate', 'Host Thumbnail Url', 'Host Picture Url', 'Host Neighbourhood', 'Host Listings Count', 'Host Total Listings Count', 'Host Verifications', 'Street', 'Neighbourhood', 'Neighbourhood Cleansed', 'Neighbourhood Group Cleansed', 'City', 'State', 'Zipcode', 'Market', 'Smart Location', 'Country Code', 'Country', 'Latitude', 'Longitude', 'Property Type', 'Room Type', 'Accommodates', 'Bathrooms', 'Bedrooms', 'Beds', 'Bed Type', 'Amenities', 'Square Feet', 'Price', 'Weekly Price', 'Monthly Price', 'Security Deposit', 'Cleaning Fee', 'Guests Included', 'Extra People', 'Minimum Nights', 'Max

Queremos quedarnos solo con los datos correspondientes a la Comunidad de Madrid

###### Filtro el dataframe para seleccionar solo aquellos datos que correspondan a Madrid. Elijo utilizar la columna 'State' para ello porque resulta más fácil debido a la sintaxis utilizada al introducir los datos.

In [293]:
uncleansed_df = uncleansed_df[uncleansed_df['State'].astype(str).str.contains('Madrid')]

print(uncleansed_df.State.unique())
print('-----------------------------------')
print(uncleansed_df.City.unique())

['Comunidad de Madrid' 'Community of Madrid' 'Madrid' 'Madrid \nMadrid'
 'Madrid, Spain' 'Região Autônoma de Madrid' 'Madrid España'
 'Communauté de Madrid' 'Madrid capital' 'Autonome Gemeinschaft Madrid'
 'España,Madrid']
-----------------------------------
['Madrid' 'Delicias-Madrid' 'madrid' 'Boadilla del Monte' 'Pozuelo'
 'San Lázaro' 'Madrid, Comunidad de Madrid, ES' 'Chueca'
 'Pozuelo de Alarcón' 'Centro, Madrid' 'MADRID' 'Madri' nan 'Aravaca'
 'San Fernando de Henares' 'Valderrodrigo' 'las matas  madrid'
 'Madrid, Comunidad de Madrid, ESPANA' 'Madrid, Vallecas (Fontarrón)'
 'Delicias' 'Aravaca (Madrid)' 'Chueca, Madrid' 'Ventas' 'Centro']


Eliminamos aquellas columnas que no nos van a resultar útiles

In [294]:
columns_to_remove = ['Listing Url', 'Scrape ID', 'Last Scraped', 'Name', 'Summary', 'Space', 'Description', 'Experiences Offered', 'Neighborhood Overview', \
                     'Notes', 'Transit', 'Access', 'Interaction', 'House Rules', 'Thumbnail Url', 'Neighbourhood Group Cleansed','Smart Location', 'Medium Url',\
                     'Picture Url', 'XL Picture Url', 'Host URL', 'Host Name', 'Host Location', 'Host About', 'Host Response Time', 'Host Response Rate',\
                     'Host Acceptance Rate', 'Host Thumbnail Url', 'Host Picture Url', 'Host Neighbourhood', 'Host Verifications', 'Market', 'Country Code', \
                     'Amenities', 'Calendar Updated', 'Has Availability', 'Calendar last Scraped', 'First Review', 'Last Review', 'License', 'Accommodates', \
                     'Jurisdiction Names', 'Reviews per Month', 'Geolocation', 'Features', 'Calculated host listings count']

df = uncleansed_df.drop(columns_to_remove, axis = 1)

print(len(df.columns))
print(df.columns.tolist())

43
['ID', 'Host ID', 'Host Since', 'Host Listings Count', 'Host Total Listings Count', 'Street', 'Neighbourhood', 'Neighbourhood Cleansed', 'City', 'State', 'Zipcode', 'Country', 'Latitude', 'Longitude', 'Property Type', 'Room Type', 'Bathrooms', 'Bedrooms', 'Beds', 'Bed Type', 'Square Feet', 'Price', 'Weekly Price', 'Monthly Price', 'Security Deposit', 'Cleaning Fee', 'Guests Included', 'Extra People', 'Minimum Nights', 'Maximum Nights', 'Availability 30', 'Availability 60', 'Availability 90', 'Availability 365', 'Number of Reviews', 'Review Scores Rating', 'Review Scores Accuracy', 'Review Scores Cleanliness', 'Review Scores Checkin', 'Review Scores Communication', 'Review Scores Location', 'Review Scores Value', 'Cancellation Policy']


Comprobamos la solidez de los datos comprobando si hay valores nulos

In [330]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13198 entries, 1021 to 492520
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID                           13198 non-null  object        
 1   Host ID                      13198 non-null  int64         
 2   Host Since                   13195 non-null  datetime64[ns]
 3   Host Listings Count          13195 non-null  float64       
 4   Street                       13198 non-null  object        
 5   Neighbourhood Cleansed       13198 non-null  object        
 6   City                         13196 non-null  object        
 7   State                        13198 non-null  object        
 8   Zipcode                      12756 non-null  object        
 9   Country                      13198 non-null  object        
 10  Latitude                     13198 non-null  float64       
 11  Longitude                    13198 no

None

###### * Podemos ver que la columna de 'Square Feet', 'Security Deposit' y 'Cleaning Fee' tienen muchos nulos pero es una información que queremos utilizar más adelante y no nos conviene eliminar las filas sin datos. 
###### * Las columnas de 'Weekly Price' y 'Monthly Price' tienen bastantes datos nulos, mientras que la columna 'Price' que indica el precio por día está completa. Tal vez resulte mejor utilizar esos datos.
###### * El resto de columnas parecen tener una proporción de datos no nulos buena.


Comprobamos que hay columnas con datos repetidos

In [296]:
df['match'] = df.apply(lambda x: x['Host Total Listings Count'] == x['Host Listings Count'], axis = 1)

print(df['match'].value_counts())

df[~df['match']][['Host Total Listings Count', 'Host Listings Count', 'match']]

True     13195
False        3
Name: match, dtype: int64


,Host Total Listings Count,Host Listings Count,match
256287,NaN,NaN,False
388849,NaN,NaN,False
492402,NaN,NaN,False


###### Decidimos quedarnos con la columna de 'Neighbourhood Cleansed' frente a 'Neighbourhood' porque no hay tantos nulos y la columna repetida que observamos anteriormente.

In [297]:
df = df.drop(['Host Total Listings Count', 'Neighbourhood', 'Weekly Price', 'Monthly Price', 'match'], axis = 1)

Normalización de ciertas columnas en las que se detectan incidencias al examinar las columnas con unique()

In [331]:
# Normalización del código postal
df['Zipcode'].unique()
replace_values = {'nan': np.nan, '-': np.nan, '28': np.nan, '-' : np.nan, '2802\n28012' : '28012', '28002\n28002': '28002', '28051\n28051' : '28051', \
                  'Madrid 28004': '28004', '2815' : '28015', '2805' : '28005'}
df = df.replace({'Zipcode': replace_values})

# Conversión del ID de la entrada en numérico en vez de string
df['ID'].astype(int)

# Conversión de las fechas de 'Host Since' en date
df['Host Since'] # El formato de fecha es YYYY-MM-DD
df['Host Since'] = pd.to_datetime(df['Host Since'])